In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, PReLU, LeakyReLU

import keras
import tensorflow as tf

from keras.callbacks import EarlyStopping

In [2]:
pd.set_option("max.columns", None)

In [3]:
df = pd.read_csv("mixed_scores_teams.csv")

In [4]:
df.head(10)

,Unnamed: 0,date,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),visitor,pts_visitor,home,pts_home,ot,home_game,back_to_back
0,0,2015-11-03,Atlanta Hawks,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,7:30p,Atlanta Hawks,98,Miami Heat,92,NaN,False,False
1,1,2015-11-04,Atlanta Hawks,5,26,12,16,3,12,16,15,24,20,27,29,17,3,26,6,8,17,16,4,7,22,12,13,6,10,22,5,20,11,8,8:00p,Brooklyn Nets,87,Atlanta Hawks,101,NaN,True,True
2,2,2015-11-06,Atlanta Hawks,3,20,10,16,3,13,15,16,25,18,26,29,15,3,24,6,6,19,11,4,10,26,12,14,6,12,17,3,18,11,8,8:00p,Atlanta Hawks,121,New Orleans Pelicans,115,NaN,False,False
3,3,2015-11-07,Atlanta Hawks,3,18,15,16,3,9,7,22,27,14,24,27,17,7,24,6,10,19,10,6,16,26,9,14,6,8,13,2,12,14,5,7:30p,Washington Wizards,99,Atlanta Hawks,114,NaN,True,True
4,4,2015-11-09,Atlanta Hawks,2,20,11,16,2,4,6,21,24,14,24,25,17,9,25,11,10,23,11,3,18,27,10,7,6,8,6,2,9,13,7,8:00p,Minnesota Timberwolves,117,Atlanta Hawks,107,NaN,True,False
5,5,2015-11-11,Atlanta Hawks,2,17,15,21,2,4,4,22,23,12,26,25,24,13,23,11,12,24,14,2,23,29,10,7,6,9,5,2,7,12,5,8:00p,New Orleans Pelicans,98,Atlanta Hawks,106,NaN,True,False
6,6,2015-11-13,Atlanta Hawks,2,15,14,19,2,3,5,22,16,13,24,23,23,10,25,7,12,19,17,2,18,30,8,6,6,7,4,2,9,11,3,7:30p,Atlanta Hawks,93,Boston Celtics,106,NaN,False,False
7,7,2015-11-15,Atlanta Hawks,2,18,16,22,2,4,8,24,17,12,23,28,20,11,25,11,15,22,18,3,15,30,7,9,11,5,6,2,12,11,3,6:00p,Utah Jazz,97,Atlanta Hawks,96,NaN,True,False
8,8,2015-11-17,Atlanta Hawks,2,19,15,27,3,4,7,25,21,12,25,26,18,15,26,11,15,22,16,2,21,30,9,8,11,6,9,2,11,12,2,7:30p,Atlanta Hawks,88,Brooklyn Nets,90,NaN,False,False
9,9,2015-11-18,Atlanta Hawks,2,16,15,25,3,3,8,24,17,11,26,24,16,12,26,12,14,26,16,3,20,30,12,8,11,11,11,2,8,17,4,8:00p,Sacramento Kings,97,Atlanta Hawks,103,NaN,True,True


In [5]:
df1 = df[(df["pts_home"] > df["pts_visitor"])]

In [6]:
base = len(df1) / len(df)
base

0.5773195876288659

In [7]:
df["won_game"] = 0

In [8]:
z = (df["home_game"] == True) & (df["pts_home"] > df["pts_visitor"])

In [9]:
df["won_game"][z] = 1 

/Users/brandonanhorn19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
z = (df["home_game"] == False) & (df["pts_visitor"] > df["pts_home"])

In [11]:
df["won_game"][z] = 1 

/Users/brandonanhorn19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df.head(10)

,Unnamed: 0,date,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),visitor,pts_visitor,home,pts_home,ot,home_game,back_to_back,won_game
0,0,2015-11-03,Atlanta Hawks,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,7:30p,Atlanta Hawks,98,Miami Heat,92,NaN,False,False,1
1,1,2015-11-04,Atlanta Hawks,5,26,12,16,3,12,16,15,24,20,27,29,17,3,26,6,8,17,16,4,7,22,12,13,6,10,22,5,20,11,8,8:00p,Brooklyn Nets,87,Atlanta Hawks,101,NaN,True,True,1
2,2,2015-11-06,Atlanta Hawks,3,20,10,16,3,13,15,16,25,18,26,29,15,3,24,6,6,19,11,4,10,26,12,14,6,12,17,3,18,11,8,8:00p,Atlanta Hawks,121,New Orleans Pelicans,115,NaN,False,False,1
3,3,2015-11-07,Atlanta Hawks,3,18,15,16,3,9,7,22,27,14,24,27,17,7,24,6,10,19,10,6,16,26,9,14,6,8,13,2,12,14,5,7:30p,Washington Wizards,99,Atlanta Hawks,114,NaN,True,True,1
4,4,2015-11-09,Atlanta Hawks,2,20,11,16,2,4,6,21,24,14,24,25,17,9,25,11,10,23,11,3,18,27,10,7,6,8,6,2,9,13,7,8:00p,Minnesota Timberwolves,117,Atlanta Hawks,107,NaN,True,False,0
5,5,2015-11-11,Atlanta Hawks,2,17,15,21,2,4,4,22,23,12,26,25,24,13,23,11,12,24,14,2,23,29,10,7,6,9,5,2,7,12,5,8:00p,New Orleans Pelicans,98,Atlanta Hawks,106,NaN,True,False,1
6,6,2015-11-13,Atlanta Hawks,2,15,14,19,2,3,5,22,16,13,24,23,23,10,25,7,12,19,17,2,18,30,8,6,6,7,4,2,9,11,3,7:30p,Atlanta Hawks,93,Boston Celtics,106,NaN,False,False,0
7,7,2015-11-15,Atlanta Hawks,2,18,16,22,2,4,8,24,17,12,23,28,20,11,25,11,15,22,18,3,15,30,7,9,11,5,6,2,12,11,3,6:00p,Utah Jazz,97,Atlanta Hawks,96,NaN,True,False,0
8,8,2015-11-17,Atlanta Hawks,2,19,15,27,3,4,7,25,21,12,25,26,18,15,26,11,15,22,16,2,21,30,9,8,11,6,9,2,11,12,2,7:30p,Atlanta Hawks,88,Brooklyn Nets,90,NaN,False,False,0
9,9,2015-11-18,Atlanta Hawks,2,16,15,25,3,3,8,24,17,11,26,24,16,12,26,12,14,26,16,3,20,30,12,8,11,11,11,2,8,17,4,8:00p,Sacramento Kings,97,Atlanta Hawks,103,NaN,True,True,1


In [13]:
df.tail(10)

,Unnamed: 0,date,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),visitor,pts_visitor,home,pts_home,ot,home_game,back_to_back,won_game
12212,398,2021-02-15,Washington Wizards,16,29,26,18,29,20,23,9,6,28,2,14,5,23,20,7,30,17,27,13,17,17,12,24,26,18,17,21,30,12,19,7:00p,Houston Rockets,119,Washington Wizards,131,NaN,True,True,1
12213,399,2021-02-17,Washington Wizards,16,30,25,18,29,21,22,9,6,28,2,14,4,24,17,7,29,17,23,13,17,17,11,24,26,18,13,17,29,12,18,8:00p,Denver Nuggets,128,Washington Wizards,130,NaN,True,False,1
12214,400,2021-02-20,Washington Wizards,13,30,26,17,30,21,22,9,7,28,1,14,3,24,17,7,30,15,25,14,19,17,11,23,27,18,11,17,27,12,18,10:00p,Washington Wizards,118,Portland Trail Blazers,111,NaN,False,False,1
12215,401,2021-02-22,Washington Wizards,14,30,25,15,30,21,22,8,10,26,1,19,3,21,21,7,29,17,24,14,15,17,10,24,28,17,11,16,29,15,18,10:00p,Washington Wizards,127,Los Angeles Lakers,124,OT,False,False,1
12216,402,2021-02-23,Washington Wizards,14,30,24,14,30,22,22,10,12,26,1,20,3,22,22,7,29,18,24,11,15,17,9,24,25,16,9,16,28,15,18,10:00p,Washington Wizards,116,Los Angeles Clippers,135,NaN,False,True,0
12217,403,2021-02-25,Washington Wizards,13,30,27,14,30,23,22,10,11,26,1,20,4,25,24,7,29,17,25,11,18,17,7,24,26,14,9,15,29,16,17,9:00p,Washington Wizards,112,Denver Nuggets,110,NaN,False,False,1
12218,404,2021-02-27,Washington Wizards,14,30,27,15,30,22,22,10,12,26,1,23,4,21,25,6,29,17,25,12,17,17,6,24,26,14,9,15,29,16,18,7:00p,Minnesota Timberwolves,112,Washington Wizards,128,NaN,True,False,1
12219,405,2021-02-28,Washington Wizards,14,30,27,14,30,22,22,10,14,26,1,25,4,21,24,6,29,18,25,12,16,17,6,24,27,13,7,15,28,17,18,7:00p,Washington Wizards,110,Boston Celtics,111,NaN,False,True,0
12220,406,2021-03-02,Washington Wizards,16,30,25,15,30,21,22,10,13,25,1,25,4,23,22,8,29,19,26,9,16,18,7,24,26,13,7,14,29,17,18,7:00p,Memphis Grizzlies,125,Washington Wizards,111,NaN,True,False,0
12221,407,2021-03-04,Washington Wizards,15,30,26,14,30,21,22,10,13,24,1,25,4,25,22,8,29,21,26,11,17,18,7,24,26,13,9,14,28,19,16,7:00p,Los Angeles Clippers,117,Washington Wizards,119,NaN,True,False,1


In [14]:
del df["Unnamed: 0"], df["date"], df["team"], df["visitor"], df["pts_visitor"], df["home"], df["pts_home"], df["ot"]

In [15]:
df.head(1)

,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),home_game,back_to_back,won_game
0,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,7:30p,False,False,1


In [16]:
df["start (et)"] = df["start (et)"].str.replace("p", "")

In [17]:
df["start (et)"] = df["start (et)"].astype(str)

In [18]:
target = "won_game"
X = df.drop(target, axis=1)
y = df[target]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [20]:
mapper = DataFrameMapper([
    (["rank_assists"], StandardScaler()),
    (["rank_blocks"], StandardScaler()),
    (["rank_deffeciency"], StandardScaler()),
    (["rank_drebounds"], StandardScaler()),
    (["rank_fouls"], StandardScaler()),
    (["rank_ft_pct"], StandardScaler()),
    (["rank_oeffeciency"], StandardScaler()),
    (["rank_opp_assists"], StandardScaler()),
    (["rank_opp_blocks"], StandardScaler()),
    (["rank_opp_drebounds"], StandardScaler()),
    (["rank_opp_fouls"], StandardScaler()),
    (["rank_opp_orebounds"], StandardScaler()),
    (["rank_opp_ptsfastbreak"], StandardScaler()),
    (["rank_opp_ptsfrom2"], StandardScaler()),
    (["rank_opp_ptsfrom3"], StandardScaler()),
    (["rank_opp_ptsinpaint"], StandardScaler()),
    (["rank_opp_ptspergame"], StandardScaler()),
    (["rank_opp_steals"], StandardScaler()),
    (["rank_opp_threepointpct"], StandardScaler()),
    (["rank_opp_turnovers"], StandardScaler()),
    (["rank_opp_twopointpct"], StandardScaler()),
    (["rank_orebounds"], StandardScaler()),
    (["rank_ptsfrom2"], StandardScaler()),
    (["rank_ptsfrom3"], StandardScaler()),
    (["rank_ptsfromfastbreak"], StandardScaler()),
    (["rank_ptsinpaint"], StandardScaler()),
    (["rank_scoring"], StandardScaler()),
    (["rank_steals"], StandardScaler()),
    (["rank_threeptpct"], StandardScaler()),
    (["rank_turnovers"], StandardScaler()),
    (["rank_twoptpct"], StandardScaler()),
    ("start (et)", LabelBinarizer()),
    ("home_game", LabelBinarizer()),
    ("back_to_back", LabelBinarizer())],df_out=True)

In [21]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [22]:
model = LogisticRegression().fit(Z_train, y_train)
print("Log regress train score is " + str(model.score(Z_train, y_train)))
print("Log regress test score is " + str(model.score(Z_test, y_test)))

Log regress train score is 0.6244817804931267
Log regress test score is 0.6017670157068062


/Users/brandonanhorn19/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
model = DecisionTreeClassifier().fit(Z_train, y_train)
print("Decision Tree train score is " + str(model.score(Z_train, y_train)))
print("Decision Tree test score is " + str(model.score(Z_test, y_test)))

Decision Tree train score is 1.0
Decision Tree test score is 0.5507198952879581


In [24]:
model = RandomForestClassifier().fit(Z_train, y_train)
print("Random Forest train score is " + str(model.score(Z_train, y_train)))
print("Random Forest test score is " + str(model.score(Z_test, y_test)))

Random Forest train score is 1.0
Random Forest test score is 0.580824607329843


In [25]:
model = CatBoostClassifier().fit(Z_train, y_train)
print("Cat Boost train score is " + str(model.score(Z_train, y_train)))
print("Cat Boost test score is " + str(model.score(Z_test, y_test)))

Learning rate set to 0.026533
0:	learn: 0.6910816	total: 55.9ms	remaining: 55.8s
1:	learn: 0.6889875	total: 59.7ms	remaining: 29.8s
2:	learn: 0.6869580	total: 63.4ms	remaining: 21.1s
3:	learn: 0.6852706	total: 67.2ms	remaining: 16.7s
4:	learn: 0.6834689	total: 70.9ms	remaining: 14.1s
5:	learn: 0.6817868	total: 74.5ms	remaining: 12.3s
6:	learn: 0.6800687	total: 78.6ms	remaining: 11.1s
7:	learn: 0.6786261	total: 82.3ms	remaining: 10.2s
8:	learn: 0.6771673	total: 86.1ms	remaining: 9.48s
9:	learn: 0.6756999	total: 89.8ms	remaining: 8.89s
10:	learn: 0.6742847	total: 93.5ms	remaining: 8.41s
11:	learn: 0.6729323	total: 97.3ms	remaining: 8.01s
12:	learn: 0.6717029	total: 103ms	remaining: 7.8s
13:	learn: 0.6703856	total: 108ms	remaining: 7.62s
14:	learn: 0.6692606	total: 114ms	remaining: 7.46s
15:	learn: 0.6681737	total: 119ms	remaining: 7.32s
16:	learn: 0.6671213	total: 125ms	remaining: 7.23s
17:	learn: 0.6660449	total: 130ms	remaining: 7.1s
18:	learn: 0.6650411	total: 136ms	remaining: 7s
19:	

176:	learn: 0.6157584	total: 884ms	remaining: 4.11s
177:	learn: 0.6156176	total: 890ms	remaining: 4.11s
178:	learn: 0.6154229	total: 895ms	remaining: 4.11s
179:	learn: 0.6152500	total: 901ms	remaining: 4.1s
180:	learn: 0.6151011	total: 906ms	remaining: 4.1s
181:	learn: 0.6150126	total: 910ms	remaining: 4.09s
182:	learn: 0.6148755	total: 914ms	remaining: 4.08s
183:	learn: 0.6146405	total: 918ms	remaining: 4.07s
184:	learn: 0.6144396	total: 923ms	remaining: 4.06s
185:	learn: 0.6141733	total: 927ms	remaining: 4.05s
186:	learn: 0.6139904	total: 930ms	remaining: 4.04s
187:	learn: 0.6137991	total: 935ms	remaining: 4.04s
188:	learn: 0.6136500	total: 939ms	remaining: 4.03s
189:	learn: 0.6134629	total: 943ms	remaining: 4.02s
190:	learn: 0.6133351	total: 946ms	remaining: 4.01s
191:	learn: 0.6132557	total: 950ms	remaining: 4s
192:	learn: 0.6130650	total: 954ms	remaining: 3.99s
193:	learn: 0.6128855	total: 958ms	remaining: 3.98s
194:	learn: 0.6126994	total: 962ms	remaining: 3.97s
195:	learn: 0.612

352:	learn: 0.5866430	total: 1.67s	remaining: 3.05s
353:	learn: 0.5864855	total: 1.67s	remaining: 3.05s
354:	learn: 0.5863133	total: 1.68s	remaining: 3.05s
355:	learn: 0.5861114	total: 1.68s	remaining: 3.04s
356:	learn: 0.5859013	total: 1.69s	remaining: 3.04s
357:	learn: 0.5857399	total: 1.69s	remaining: 3.04s
358:	learn: 0.5856207	total: 1.7s	remaining: 3.03s
359:	learn: 0.5854936	total: 1.7s	remaining: 3.02s
360:	learn: 0.5852742	total: 1.71s	remaining: 3.02s
361:	learn: 0.5850554	total: 1.71s	remaining: 3.01s
362:	learn: 0.5848848	total: 1.71s	remaining: 3.01s
363:	learn: 0.5846606	total: 1.72s	remaining: 3s
364:	learn: 0.5844694	total: 1.72s	remaining: 2.99s
365:	learn: 0.5842783	total: 1.72s	remaining: 2.99s
366:	learn: 0.5841422	total: 1.73s	remaining: 2.98s
367:	learn: 0.5838943	total: 1.73s	remaining: 2.97s
368:	learn: 0.5836786	total: 1.74s	remaining: 2.97s
369:	learn: 0.5835115	total: 1.74s	remaining: 2.96s
370:	learn: 0.5833115	total: 1.74s	remaining: 2.96s
371:	learn: 0.583

528:	learn: 0.5548713	total: 2.41s	remaining: 2.15s
529:	learn: 0.5547016	total: 2.42s	remaining: 2.14s
530:	learn: 0.5545548	total: 2.42s	remaining: 2.14s
531:	learn: 0.5543572	total: 2.43s	remaining: 2.13s
532:	learn: 0.5540988	total: 2.43s	remaining: 2.13s
533:	learn: 0.5539172	total: 2.44s	remaining: 2.13s
534:	learn: 0.5537042	total: 2.44s	remaining: 2.12s
535:	learn: 0.5534987	total: 2.44s	remaining: 2.12s
536:	learn: 0.5533080	total: 2.45s	remaining: 2.11s
537:	learn: 0.5531350	total: 2.45s	remaining: 2.11s
538:	learn: 0.5529898	total: 2.46s	remaining: 2.1s
539:	learn: 0.5527387	total: 2.46s	remaining: 2.1s
540:	learn: 0.5525642	total: 2.46s	remaining: 2.09s
541:	learn: 0.5524251	total: 2.47s	remaining: 2.08s
542:	learn: 0.5522831	total: 2.47s	remaining: 2.08s
543:	learn: 0.5521048	total: 2.48s	remaining: 2.08s
544:	learn: 0.5520013	total: 2.48s	remaining: 2.07s
545:	learn: 0.5518591	total: 2.48s	remaining: 2.06s
546:	learn: 0.5516749	total: 2.49s	remaining: 2.06s
547:	learn: 0.

693:	learn: 0.5285904	total: 3.08s	remaining: 1.36s
694:	learn: 0.5284294	total: 3.09s	remaining: 1.36s
695:	learn: 0.5283295	total: 3.1s	remaining: 1.35s
696:	learn: 0.5282125	total: 3.1s	remaining: 1.35s
697:	learn: 0.5280339	total: 3.11s	remaining: 1.34s
698:	learn: 0.5279154	total: 3.11s	remaining: 1.34s
699:	learn: 0.5277460	total: 3.11s	remaining: 1.33s
700:	learn: 0.5276477	total: 3.12s	remaining: 1.33s
701:	learn: 0.5274581	total: 3.12s	remaining: 1.32s
702:	learn: 0.5273096	total: 3.13s	remaining: 1.32s
703:	learn: 0.5271694	total: 3.13s	remaining: 1.32s
704:	learn: 0.5269945	total: 3.14s	remaining: 1.31s
705:	learn: 0.5268238	total: 3.14s	remaining: 1.31s
706:	learn: 0.5267038	total: 3.14s	remaining: 1.3s
707:	learn: 0.5266094	total: 3.15s	remaining: 1.3s
708:	learn: 0.5264949	total: 3.15s	remaining: 1.29s
709:	learn: 0.5263528	total: 3.15s	remaining: 1.29s
710:	learn: 0.5262179	total: 3.16s	remaining: 1.28s
711:	learn: 0.5260889	total: 3.16s	remaining: 1.28s
712:	learn: 0.52

860:	learn: 0.5052788	total: 3.78s	remaining: 611ms
861:	learn: 0.5051268	total: 3.79s	remaining: 606ms
862:	learn: 0.5049688	total: 3.79s	remaining: 602ms
863:	learn: 0.5047950	total: 3.8s	remaining: 598ms
864:	learn: 0.5046695	total: 3.8s	remaining: 594ms
865:	learn: 0.5044845	total: 3.81s	remaining: 589ms
866:	learn: 0.5043056	total: 3.81s	remaining: 585ms
867:	learn: 0.5041778	total: 3.82s	remaining: 580ms
868:	learn: 0.5040015	total: 3.82s	remaining: 576ms
869:	learn: 0.5039044	total: 3.82s	remaining: 571ms
870:	learn: 0.5037221	total: 3.83s	remaining: 567ms
871:	learn: 0.5036348	total: 3.83s	remaining: 562ms
872:	learn: 0.5035372	total: 3.83s	remaining: 558ms
873:	learn: 0.5034321	total: 3.84s	remaining: 553ms
874:	learn: 0.5032447	total: 3.84s	remaining: 549ms
875:	learn: 0.5030655	total: 3.85s	remaining: 545ms
876:	learn: 0.5028593	total: 3.85s	remaining: 540ms
877:	learn: 0.5027052	total: 3.85s	remaining: 536ms
878:	learn: 0.5025586	total: 3.86s	remaining: 531ms
879:	learn: 0.

In [26]:
y_pred = model.predict(Z_test)

In [27]:
confusion_matrix(y_test, y_pred)

array([[896, 568],
       [636, 956]])

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.61      0.60      1464
           1       0.63      0.60      0.61      1592

    accuracy                           0.61      3056
   macro avg       0.61      0.61      0.61      3056
weighted avg       0.61      0.61      0.61      3056



In [29]:
# Deep Learning

In [30]:
model = Sequential([
    Input(shape=(Z_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="relu")
])

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

early_stop = EarlyStopping(monitor="val_loss", patience=7, verbose=0, mode="min")

model.fit(Z_train, y_train,
            validation_data=(Z_test, y_test),
            epochs=200, batch_size=8,
            verbose=2, callbacks=[early_stop])

Epoch 1/200
1146/1146 - 2s - loss: 0.8136 - accuracy: 0.5664 - val_loss: 0.7205 - val_accuracy: 0.5628
Epoch 2/200
1146/1146 - 1s - loss: 0.6669 - accuracy: 0.6072 - val_loss: 0.6726 - val_accuracy: 0.5998
Epoch 3/200
1146/1146 - 1s - loss: 0.6570 - accuracy: 0.6258 - val_loss: 0.6689 - val_accuracy: 0.5975
Epoch 4/200
1146/1146 - 1s - loss: 0.6563 - accuracy: 0.6256 - val_loss: 0.6679 - val_accuracy: 0.6001
Epoch 5/200
1146/1146 - 1s - loss: 0.6456 - accuracy: 0.6344 - val_loss: 0.6812 - val_accuracy: 0.5805
Epoch 6/200
1146/1146 - 1s - loss: 0.6352 - accuracy: 0.6440 - val_loss: 0.6668 - val_accuracy: 0.5975
Epoch 7/200
1146/1146 - 1s - loss: 0.6304 - accuracy: 0.6519 - val_loss: 0.6671 - val_accuracy: 0.5991
Epoch 8/200
1146/1146 - 1s - loss: 0.6298 - accuracy: 0.6534 - val_loss: 0.6880 - val_accuracy: 0.5890
Epoch 9/200
1146/1146 - 1s - loss: 0.6209 - accuracy: 0.6600 - val_loss: 0.6886 - val_accuracy: 0.5815
Epoch 10/200
1146/1146 - 1s - loss: 0.6206 - accuracy: 0.6638 - val_loss:

In [31]:
# Boston - apr. 25, 2021 - scored 104
X_new = pd.DataFrame({
    'back_to_back': [False],
    'home_game': [False],
    'rank_assists': [23],
    'rank_blocks': [10],
    'rank_deffeciency': [12],
    'rank_drebounds': [21],
    'rank_fouls': [22],
    'rank_ft_pct': [17],
    'rank_oeffeciency': [12],
    'rank_opp_assists': [7],
    'rank_opp_blocks': [10],
    'rank_opp_drebounds': [5],
    'rank_opp_fouls': [15],
    'rank_opp_orebounds': [11],
    'rank_opp_ptsfastbreak': [12],
    'rank_opp_ptsfrom2': [5],
    'rank_opp_ptsfrom3': [16],
    'rank_opp_ptsinpaint': [6],
    'rank_opp_ptspergame': [9],
    'rank_opp_steals': [11],
    'rank_opp_threepointpct': [17],
    'rank_opp_turnovers': [14],
    'rank_opp_twopointpct': [13],
    'rank_orebounds': [5],
    'rank_ptsfrom2': [15],
    'rank_ptsfrom3': [11],
    'rank_ptsfromfastbreak': [18],
    'rank_ptsinpaint': [19],
    'rank_scoring': [16],
    'rank_steals': [12],
    'rank_threeptpct': [9],
    'rank_turnovers': [17],
    'rank_twoptpct': [14],
    'start (et)': ['1:00']
})

Z_new = mapper.transform(X_new)
model.predict(Z_new)

array([[0.3284043]], dtype=float32)

In [32]:
# Charlotte - apr. 25, 2021 - scored 125
X_new = pd.DataFrame({
    'back_to_back': [False],
    'home_game': [True],
    'rank_assists': [8],
    'rank_blocks': [18],
    'rank_deffeciency': [17],
    'rank_drebounds': [24],
    'rank_fouls': [4],
    'rank_ft_pct': [21],
    'rank_oeffeciency': [18],
    'rank_opp_assists': [30],
    'rank_opp_blocks': [13],
    'rank_opp_drebounds': [17],
    'rank_opp_fouls': [24],
    'rank_opp_orebounds': [27],
    'rank_opp_ptsfastbreak': [24],
    'rank_opp_ptsfrom2': [4],
    'rank_opp_ptsfrom3': [30],
    'rank_opp_ptsinpaint': [13],
    'rank_opp_ptspergame': [13],
    'rank_opp_steals': [24],
    'rank_opp_threepointpct': [16],
    'rank_opp_turnovers': [10],
    'rank_opp_twopointpct': [30],
    'rank_orebounds': [10],
    'rank_ptsfrom2': [25],
    'rank_ptsfrom3': [8],
    'rank_ptsfromfastbreak': [10],
    'rank_ptsinpaint': [21],
    'rank_scoring': [22],
    'rank_steals': [10],
    'rank_threeptpct': [8],
    'rank_turnovers': [26],
    'rank_twoptpct': [21],
    'start (et)': ['1:00']
})

Z_new = mapper.transform(X_new)
model.predict(Z_new)

array([[0.7114772]], dtype=float32)